# Imports

In [ ]:
import numpy as np
import scipy.linalg as spla
import scipy.io as spio
import matplotlib.pyplot as plt

from pymor.models.iosys import LTIModel

In [ ]:
plt.rcParams['figure.dpi'] = 100
plt.rcParams['axes.grid'] = True

# Build model

Consider the $2 \times 2$ thermal block problem in the state-space representation

\begin{align}
E \dot{x}(t) &= A x(t) + B u(t), \\
y(t) &= C x(t),
\end{align}

with $A, E \in \mathbb{R}^{7488 \times 7488}$, $B \in \mathbb{R}^{7488 \times 1}$ and $C \in \mathbb{R}^{4 \times 7488}$. 

In [ ]:
mat = spio.loadmat('data/ABCE.mat')
mat.keys()

In [ ]:
mu = np.sqrt(10) * np.array([0.2, 0.4, 0.6, 0.8])
A = mat['A0']
for i in range(4):
    A += mu[i] * mat[f'A{i + 1}']
B = mat['B']
C = mat['C']
E = mat['E']

In [ ]:
fom = LTIModel.from_matrices(A, B, C, E=E)

In [ ]:
fom

In [ ]:
print(fom)

# Bode plot

In [ ]:
w = np.logspace(-2, 4, 100)
fig, axs = plt.subplots(2, 1, figsize=(6, 8), constrained_layout=True)
_ = fom.bode_plot(w, ax=np.array(4 * [[axs[0]], [axs[1]]]))

In [ ]:
_ = fom.mag_plot(w)

# Hankel singular values

In [ ]:
hsv = fom.hsv()

In [ ]:
fig, ax = plt.subplots()
_ = ax.semilogy(range(1, len(hsv) + 1), hsv, '.-')

# Balanced truncation

Solving the Lyapunov equations

\begin{align}
    A \mathcal{P} E^{\operatorname{T}} + E \mathcal{P} A^{\operatorname{T}} + B B^{\operatorname{T}} &= 0, \\
    A^{\operatorname{T}} \mathcal{P} E + E^{\operatorname{T}} \mathcal{P} A + C^{\operatorname{T}} C &= 0
\end{align}

and computing projection matrices for balanced truncation is all handled by the **BTReductor** class.

In [ ]:
from pymor.reductors.bt import BTReductor

In [ ]:
bt = BTReductor(fom)

In [ ]:
fig, ax = plt.subplots()
error_bounds = bt.error_bounds()
_ = ax.semilogy(range(1, len(error_bounds) + 1), error_bounds, '.-')

In [ ]:
hinf_norm = spla.norm(fom.eval_tf(0))
hinf_norm

In [ ]:
rom = bt.reduce(tol=1e-5)

In [ ]:
rom

In [ ]:
print(rom)

# Error system

The difference of transfer functions of two LTI systems

\begin{align}
H(s) - \hat{H}(s) = C (s E - A)^{-1} B - \hat{C} (s \hat{E} - \hat{A})^{-1} \hat{B}
\end{align}

can be represented as

\begin{align}
H(s) - \hat{H}(s) = \begin{bmatrix} C & -\hat{C} \end{bmatrix} \left(s \begin{bmatrix} E & 0 \\ 0 & \hat{E}\end{bmatrix} - \begin{bmatrix} A & 0 \\ 0 & \hat{A} \end{bmatrix} \right)^{-1} \begin{bmatrix} B \\ \hat{B}\end{bmatrix}.
\end{align}

In [ ]:
err = fom - rom

In [ ]:
err

In [ ]:
print(err)

# Bode plot of the error system

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(6, 8), constrained_layout=True)
_ = err.bode_plot(w, ax=np.array(4 * [[axs[0]], [axs[1]]]))

In [ ]:
_ = err.mag_plot(w)